In [ ]:
# ==============================
# STEP 1: Install dependencies
# ==============================
!pip install -q transformers datasets librosa pydub langchain nltk

# ==============================
# STEP 2: Imports
# ==============================
import os
import librosa
import torch
import nltk
from pydub import AudioSegment
from transformers import pipeline
from langchain import LLMChain, PromptTemplate
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from google.colab import files

# Download VADER lexicon
nltk.download("vader_lexicon")

# ==============================
# STEP 3: Upload MP3
# ==============================
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Convert MP3 to WAV
if file_name.endswith(".mp3"):
    wav_path = file_name.replace(".mp3", ".wav")
    audio = AudioSegment.from_mp3(file_name)
    audio.export(wav_path, format="wav")
else:
    wav_path = file_name

# ==============================
# STEP 4: Transcribe with Whisper
# ==============================
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_id = "openai/whisper-small"  # small = free + decent
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id).to(device)

# Load audio
speech, sr = librosa.load(wav_path, sr=16000)
inputs = processor(speech, sampling_rate=16000, return_tensors="pt").input_features.to(device)

# Generate transcription
with torch.no_grad():
    predicted_ids = model.generate(inputs)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print("===== TRANSCRIPTION =====")
print(transcription)

# ==============================
# STEP 5: Summarization
# ==============================
from transformers import pipeline

# Hugging Face summarizer (free)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

summary = summarizer(
    transcription,
    max_length=200,
    min_length=50,
    do_sample=False
)[0]["summary_text"]

print("\n===== SUMMARY =====")
print(summary)

# ==============================
# STEP 6: Sentiment Analysis
# ==============================
sia = SentimentIntensityAnalyzer()
sentiment = sia.polarity_scores(transcription)

print("\n===== SENTIMENT ANALYSIS =====")
print(sentiment)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Saving Hi, I’m having .mp3 to Hi, I’m having .mp3
===== TRANSCRIPTION =====
 Hi, and having trouble logging into your app. Every time I enter my password, it says invalid credentials, even though I reset it just this morning.I'm really sorry you're facing this issue. Let's get it sorted right away. Can you please share the phone number or email address associated with your account? Yes, my number is 99876-54321. Thank you. Let me quickly check the login activity on your account.


Device set to use cpu
Your max_length is set to 200, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)



===== SUMMARY =====
Every time I enter my password, it says invalid credentials, even though I reset it just this morning. Can you please share the phone number or email address associated with your account? Yes, my number is 99876-54321. Thank you.

===== SENTIMENT ANALYSIS =====
{'neg': 0.057, 'neu': 0.78, 'pos': 0.163, 'compound': 0.719}
